In [3]:
#imports
import spatialdata
import dask_image.imread
import dask.array as da
import geopandas as gpd
import anndata as ad
import napari_spatialdata
import time

import sys, os
sys.path.append(os.path.abspath("../functions/"))
import anndata_utils

for package in [spatialdata, gpd, ad, napari_spatialdata]:
    print(f"{package.__name__}: {package.__version__}")

spatialdata: 0.2.3
geopandas: 1.0.1
anndata: 0.11.3
napari_spatialdata: 0.5.3


In [4]:
#paths
path_to_image = "../data/exemplar001/registration/Exemplar001.ome.tif"
path_to_mask = "../data/exemplar001/segmentation/mesmer-Exemplar001/cell.tif"
path_to_quant = "../data/exemplar001/quantification/Exemplar001--mesmer_cell.csv" 

In [5]:
#create sdata
sdata = spatialdata.SpatialData()

In [6]:
#parse mIF image
image = dask_image.imread.imread(path_to_image)
sdata['image'] = spatialdata.models.Image2DModel.parse(image)

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


In [9]:
#parse mask
mask = dask_image.imread.imread(path_to_mask)
mask = da.squeeze(mask) #remove singleton dimension
sdata['mask'] = spatialdata.models.Labels2DModel.parse(mask)

INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


In [10]:
#parse quantification table
adata = anndata_utils.read_quant(path_to_quant) #create adata from csv
sdata["imaging"] = spatialdata.models.TableModel.parse(adata)

2025-01-14 13:35:10.359 | INFO     | anndata_utils:read_quant:13 -  ---- read_quant : version number 1.1.0 ----
2025-01-14 13:35:10.422 | INFO     | anndata_utils:read_quant:33 -  9711 cells and 12 variables
2025-01-14 13:35:10.423 | INFO     | anndata_utils:read_quant:34 -  ---- read_quant is done, took 0s  ----


In [11]:
sdata

SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (12, 3139, 2511)
├── Labels
│     └── 'mask': DataArray[yx] (3139, 2511)
└── Tables
      └── 'imaging': AnnData (9711, 12)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels)

In [13]:
#write as zarr object
sdata.write("../data/exemplar001/exemplar001.zarr")

INFO     The Zarr backing store has been changed from None the new file path: ../data/exemplar001/exemplar001.zarr 


In [14]:
# QC1 Visual check of stitching and segmentation
interactive = napari_spatialdata.Interactive(sdata)
interactive.run()